# Extracting nos operon from genomes
Scripts to extract operons from genbank files and also retrieve gene sequences into one fasta file for tree building

In [1]:
from Bio import SeqIO
from Bio.SeqRecord import SeqRecord
from Bio.SeqFeature import SeqFeature, FeatureLocation
import os

In [2]:
def get_genbank_files(directory, extension=".gbff"):
    """Recursively retrieve all GenBank files with the specified extension from a directory and its subdirectories."""
    genbank_files = []
    for root, _, files in os.walk(directory):
        for file in files:
            if file.endswith(extension):
                genbank_files.append(os.path.join(root, file))
    return genbank_files

def extract_operon(genbank_files, reference_genes, flanking_size=15000, output_dir="extracted_operons"):
    """
    Extracts the operon plus 10kb flanking regions from GenBank files.
    Saves the extracted region as a new GenBank file with corrected feature coordinates.
    
    Parameters:
    - genbank_files: List of paths to GenBank files.
    - reference_genes: List of keywords to match within gene names or product descriptions.
    - flanking_size: Size of flanking region to include (default 10kb).
    - output_dir: Directory to save extracted GenBank files.
    """
    
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    for gbk_file in genbank_files:
        counter = 0
        for record in SeqIO.parse(gbk_file, "genbank"):
            operon_start, operon_end = None, None
            
            for feature in record.features:
                if feature.type == "CDS":
                    gene_name = feature.qualifiers.get("gene", [""])[0]
                    product_name = feature.qualifiers.get("product", [""])[0]
                    
                    if any(keyword in gene_name for keyword in reference_genes) or any(keyword in product_name for keyword in reference_genes):
                        print(product_name)
                        print(feature.location.start.position)
                        print(operon_end)
                        if (operon_end is not None) and (max(0, feature.location.start.position) < operon_end):
                            continue #avoids double counting operon that was already exported due to gene name similarities
                        operon_start = max(0, feature.location.start.position - flanking_size)
                        operon_end = min(len(record.seq), feature.location.end.position + flanking_size)
            
                        extracted_seq = record.seq[operon_start:operon_end]
                        
                        extracted_features = []
                        for f in record.features:
                            if f.location and operon_start <= f.location.start.position and f.location.end.position <= operon_end:
                                new_location = FeatureLocation(f.location.start.position - operon_start, f.location.end.position - operon_start, strand=f.location.strand)
                                new_feature = SeqFeature(new_location, type=f.type, qualifiers=f.qualifiers)
                                extracted_features.append(new_feature)
                        
                        new_record = SeqRecord(
                            extracted_seq,
                            id=record.id,
                            name=record.name,
                            description=f"Extracted operon from {record.id}, including {flanking_size}bp flanking regions",
                            annotations=record.annotations
                        )
                        new_record.features = extracted_features
                        
                        output_file = os.path.join(output_dir, f"{record.description}_{operon_start}_operon.gbk")
                        with open(output_file, "w") as out_handle:
                            SeqIO.write(new_record, out_handle, "genbank")
                        
                        print(f"Saved extracted operon to {output_file}")
                        counter += 1
            if counter == 0:
                print(f"No matching reference gene found in {gbk_file}")


In [3]:
def extract_genes(genbank_files, reference_genes, output_dir="extracted_genes"):
    """
    Extracts the specified gene sequence from a given list of genbank files
    """
    
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    gene = reference_genes[0].replace(" ",'_')
    output_file = os.path.join(output_dir, f"{gene}.fasta")
    counter = True
    for gbk_file in genbank_files:
        for record in SeqIO.parse(gbk_file, "genbank"):
            
            for feature in record.features:
                if feature.type == "CDS":
                    gene_name = feature.qualifiers.get("gene", [""])[0]
                    product_name = feature.qualifiers.get("product", [""])[0]
                    
                    if any(keyword in gene_name for keyword in reference_genes) or any(keyword in product_name for keyword in reference_genes):
                        translation = feature.qualifiers.get("translation")[0]
                        if counter:
                            with open(output_file, "w") as out_handle:
                                out_handle.write(">"+product_name+' ['+gbk_file.split('/')[-1][0:-4]+']\n')
                                out_handle.write(translation+'\n')
                            counter = False
                        else:
                            with open(output_file, "a") as out_handle:
                                out_handle.write(">"+product_name+' ['+gbk_file.split('/')[-1][0:-4]+']\n')
                                out_handle.write(translation+'\n')

In [4]:
# Example usage
directory = "./data"
genbank_files = get_genbank_files(directory)
reference_gene = ["nitrous-oxide reductase","Nitrous-oxide reductase","nitrous oxide reductase", "Nitrous oxide reductase"]  # Change this to a known gene in the operon
extract_operon(genbank_files, reference_gene, output_dir="extracted_operons/nos_all")

Nitrous oxide reductase maturation protein, outer-membrane lipoprotein NosL
383091
None
Saved extracted operon to extracted_operons/Nitrobacter vulgaris MLSD-S22_368091_operon.gbk
Nitrous oxide reductase maturation transmembrane protein NosY
383642
398646
Nitrous oxide reductase maturation protein NosF (ATPase)
384466
398646
Nitrous oxide reductase maturation protein NosD
385383
398646
Nitrous-oxide reductase (EC 1.7.99.6)
386756
398646
Nitrous oxide reductase maturation protein NosR
388728
398646
No matching reference gene found in ./data/GCA_000012725.1/genomic.gbff
No matching reference gene found in ./data/GCA_000013885.1/genomic.gbff
No matching reference gene found in ./data/GCA_000013885.1/genomic.gbff
No matching reference gene found in ./data/GCA_000013885.1/genomic.gbff
No matching reference gene found in ./data/GCA_000013885.1/genomic.gbff
Nitrous-oxide reductase
1738971
None
Saved extracted operon to extracted_operons/Anaeromyxobacter dehalogenans 2CP-1 chromosome, complete

In [6]:
directory = "./extracted_operons/nos_subset" # Use subset of operons after manual curation
genbank_files = get_genbank_files(directory, extension='.gbk')
reference_gene = ["nitrous-oxide reductase", "Nitrous-oxide reductase"]
extract_genes(genbank_files, reference_gene, output_dir="extracted_genes")

In [7]:
# Example usage
directory = "./data"
genbank_files = get_genbank_files(directory)
reference_gene = ["nitrate reductase subunit alpha", "nitrate reductase alpha"]  # Change this to a known gene in the operon
extract_operon(genbank_files, reference_gene, output_dir='extracted_operons/nitrate_reductase_alpha')

Respiratory nitrate reductase alpha chain (EC 1.7.99.4)
114565
None
Saved extracted operon to extracted_operons/nitrate_reductase_alpha/Nitrobacter vulgaris MLSD-S22_99565_operon.gbk
Respiratory nitrate reductase alpha chain (EC 1.7.99.4)
1806755
133210
Saved extracted operon to extracted_operons/nitrate_reductase_alpha/Nitrobacter vulgaris MLSD-S22_1791755_operon.gbk
No matching reference gene found in ./data/GCA_000012725.1/genomic.gbff
No matching reference gene found in ./data/GCA_000013885.1/genomic.gbff
No matching reference gene found in ./data/GCA_000013885.1/genomic.gbff
No matching reference gene found in ./data/GCA_000013885.1/genomic.gbff
No matching reference gene found in ./data/GCA_000013885.1/genomic.gbff
No matching reference gene found in ./data/GCA_000022145.1/genomic.gbff
No matching reference gene found in ./data/GCA_000519045.1/genomic.gbff
No matching reference gene found in ./data/GCA_001440475.1/genomic.gbff
No matching reference gene found in ./data/GCA_001440

In [8]:
directory = "./extracted_operons/nitrate_reductase_alpha_subset" # Use subset of sequences after manual curation
genbank_files = get_genbank_files(directory, extension='.gbk')
reference_gene = ["nitrate reductase subunit alpha", "nitrate reductase alpha"]
extract_genes(genbank_files, reference_gene)

## Extract 16S rRNA genes for tree building

In [9]:
def extract_16s_rrna(genbank_files, output_fasta="16s_rrna_sequences.fasta"):
    """
    Extract 16S rRNA sequences from a list of GenBank files and save them to a FASTA file.

    Parameters:
    - genbank_files: List of paths to GenBank files.
    - output_fasta: Path to save the extracted 16S rRNA sequences as a FASTA file.
    """
    with open(output_fasta, "w") as fasta_out:
        for gbk_file in genbank_files:
            for record in SeqIO.parse(gbk_file, "genbank"):
                for feature in record.features:
                    if feature.type == "rRNA":
                        product_name = feature.qualifiers.get("product", [""])[0]
                        if "16S" in product_name:  # Check if it's a 16S rRNA gene
                            sequence = feature.location.extract(record.seq)
                            seq_id = f"{record.description}_{feature.location.start}_{feature.location.end}"
                            fasta_out.write(f">{seq_id} {product_name}\n{sequence}\n")

    print(f"Extracted 16S rRNA sequences saved to {output_fasta}")

In [10]:
directory = "./data"
genbank_files = get_genbank_files(directory)
extract_16s_rrna(genbank_files, output_fasta="extracted_genes/16s_rrna_sequences.fasta")

Extracted 16S rRNA sequences saved to extracted_genes/16s_rrna_sequences.fasta


In [11]:
def remove_duplicates_fasta(input_fasta, output_fasta):
    """
    Reads a FASTA file and removes duplicate entries based on both sequence ID and sequence content.
    
    Parameters:
    - input_fasta: Path to the input FASTA file.
    - output_fasta: Path to save the cleaned FASTA file with unique entries.
    """
    unique_entries = set()
    unique_records = []

    # Read and store unique sequences based on (ID, Sequence)
    for record in SeqIO.parse(input_fasta, "fasta"):
        entry_key = record.description  # Tuple (header, sequence)
        if entry_key not in unique_entries:
            unique_entries.add(entry_key)
            unique_records.append(record)

    # Write unique sequences to a new file
    with open(output_fasta, "w") as out_fasta:
        SeqIO.write(unique_records, out_fasta, "fasta")

    print(f"Removed duplicates: {input_fasta} -> {output_fasta}")
    print(f"Total unique entries: {len(unique_records)}")

# Example Usage
#input_fasta = "16s_rrna_sequences.fasta"
#output_fasta = "unique_16s_rrna_sequences.fasta"
#remove_duplicates_fasta(input_fasta, output_fasta)


In [12]:
input_fasta = "extracted_genes/16s_rrna_sequences.fasta"
output_fasta = "extracted_genes/unique_16s_rrna_sequences.fasta"
remove_duplicates_fasta(input_fasta, output_fasta)

Removed duplicates: extracted_genes/16s_rrna_sequences.fasta -> extracted_genes/unique_16s_rrna_sequences.fasta
Total unique entries: 84
